<a href="https://colab.research.google.com/github/tojangeng262/connect4games/blob/main/testwithnewsimbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import json
import random
import time
import datetime
import copy



class Connect4Game:
    def __init__(self, rows=6, columns=7):
        """
        Initialize the Connect 4 board and game state.
        - rows: Number of rows (default 6).
        - columns: Number of columns (default 7).
        """
        self.rows = rows
        self.columns = columns
        # Create a rows x columns board initialized with zeros (empty cells)
        self.board = [[0 for _ in range(columns)] for _ in range(rows)]
        # List to store move notation for analysis
        self.move_log = []
        # Player 1 starts (we use 1 and 2 to represent players)
        self.current_player = 1
        # Game state
        self.game_over = False

    def is_valid_move(self, column):
        """Return True if the top cell in the column is empty (i.e., the move is valid)."""
        return self.board[0][column] == 0

    def get_next_open_row(self, column):
        """
        Return the next open row in the specified column.
        We fill from the bottom of the board upward.
        """
        for r in range(self.rows-1, -1, -1):
            if self.board[r][column] == 0:
                return r
        return None

    def drop_piece(self, row, column, piece):
        """Place a piece (1 or 2) in the board at the specified row and column."""
        self.board[row][column] = piece

    def make_move(self, column):
        """
        Attempt to make a move in the specified column.
        Returns True if move is successful; False if the column is full or invalid.
        Records the move in the move_log.
        """
        if column < 0 or column >= self.columns:
            raise ValueError("Column index out of range.")

        if not self.is_valid_move(column):
            print(f"Column {column} is full. Move invalid.")
            return False

        row = self.get_next_open_row(column)
        if row is None:
            print(f"No open row found in column {column}.")
            return False

        # Drop the piece on the board.
        self.drop_piece(row, column, self.current_player)

        # Record move notation
        move_entry = {
            "move_number": len(self.move_log) + 1,
            "player": self.current_player,
            "column": column,
            "row": row,
            "timestamp": datetime.datetime.now().isoformat()
        }
        self.move_log.append(move_entry)

        # Check for a win after the move.
        if self.winning_move(self.current_player):
            self.game_over = True
            print(f"Player {self.current_player} wins!")

        # Switch player
        self.current_player = 2 if self.current_player == 1 else 1

        return True

    def winning_move(self, piece):
        """Check whether the current board has a winning move for the given piece."""
        # Check horizontal locations for win
        for c in range(self.columns - 3):
            for r in range(self.rows):
                if (self.board[r][c] == piece and self.board[r][c+1] == piece and
                    self.board[r][c+2] == piece and self.board[r][c+3] == piece):
                    return True

        # Check vertical locations for win
        for c in range(self.columns):
            for r in range(self.rows - 3):
                if (self.board[r][c] == piece and self.board[r+1][c] == piece and
                    self.board[r+2][c] == piece and self.board[r+3][c] == piece):
                    return True

        # Check positively sloped diagonals
        for c in range(self.columns - 3):
            for r in range(self.rows - 3):
                if (self.board[r][c] == piece and self.board[r+1][c+1] == piece and
                    self.board[r+2][c+2] == piece and self.board[r+3][c+3] == piece):
                    return True

        # Check negatively sloped diagonals
        for c in range(self.columns - 3):
            for r in range(3, self.rows):
                if (self.board[r][c] == piece and self.board[r-1][c+1] == piece and
                    self.board[r-2][c+2] == piece and self.board[r-3][c+3] == piece):
                    return True

        return False

    def print_board(self):
        """Print the current board state. (Row 0 is the top of the board)"""
        for row in self.board:
            print(row)
        print("\n")

    def save_game_log(self, filename="connect4_game_log.json"):
        """
        Save the recorded move log to a JSON file.
        This file can later be used for analysis.
        """
        with open(filename, "w") as f:
            json.dump(self.move_log, f, indent=4)
        print(f"Game log saved to {filename}")


class Bot:
    def __init__(self, player_id):
        """
        Initialize the bot with its player id (1 or 2).
        """
        self.player_id = player_id

    def choose_move(self, game):
        """
        Given the current game state, return a valid column number.
        This method should be overridden by specific bot implementations.
        """
        raise NotImplementedError("This method should be implemented by subclasses.")

    def format_move(self, game, column):
        """
        Given the chosen column and game state, determine the row where the piece will land.
        Return a move dictionary in the standardized format.
        """
        row = game.get_next_open_row(column)
        move_entry = {
            "move_number": len(game.move_log) + 1,
            "player": self.player_id,
            "column": column,
            "row": row,
            "timestamp": datetime.datetime.now().isoformat()
        }
        return move_entry

class RandomBot(Bot):
    def choose_move(self, game):
        """
        Chooses a random valid move from the available columns.
        Returns:
            An integer representing the chosen column (0-indexed).
        """
        # Get a list of valid columns (where the top cell is still empty)
        valid_columns = [col for col in range(game.columns) if game.is_valid_move(col)]

        if not valid_columns:
            raise Exception("No valid moves available!")

        # Randomly choose one of the valid columns.
        chosen_column = random.choice(valid_columns)
        return chosen_column


class HeuristicBot(Bot):
    def _get_next_open_row(self, board, col):
        """
        Given a board state (a 2D list) and a column,
        return the next open row (from bottom) in that column.
        """
        for r in range(len(board) - 1, -1, -1):
            if board[r][col] == 0:
                return r
        return None

    def _winning_move(self, board, piece, rows, columns):
        """
        Check if placing a piece on the given board results in a win.
        This function mirrors the winning_move checks in the game class.
        """
        # Check horizontal locations for win
        for r in range(rows):
            for c in range(columns - 3):
                if (board[r][c] == piece and board[r][c+1] == piece and
                    board[r][c+2] == piece and board[r][c+3] == piece):
                    return True

        # Check vertical locations for win
        for r in range(rows - 3):
            for c in range(columns):
                if (board[r][c] == piece and board[r+1][c] == piece and
                    board[r+2][c] == piece and board[r+3][c] == piece):
                    return True

        # Check positively sloped diagonals
        for r in range(rows - 3):
            for c in range(columns - 3):
                if (board[r][c] == piece and board[r+1][c+1] == piece and
                    board[r+2][c+2] == piece and board[r+3][c+3] == piece):
                    return True

        # Check negatively sloped diagonals
        for r in range(3, rows):
            for c in range(columns - 3):
                if (board[r][c] == piece and board[r-1][c+1] == piece and
                    board[r-2][c+2] == piece and board[r-3][c+3] == piece):
                    return True

        return False

    def choose_move(self, game):
        """
        Implements the heuristic decision-making:
        1. Check for an immediate winning move.
        2. Check for an immediate move to block the opponent.
        3. Prefer the center column if available.
        4. Otherwise, choose a column closest to the center.
        Returns a valid column number.
        """
        valid_columns = [col for col in range(game.columns) if game.is_valid_move(col)]

        # 1. Check for an immediate winning move.
        for col in valid_columns:
            board_copy = [row[:] for row in game.board]  # deep copy of the board
            row = self._get_next_open_row(board_copy, col)
            if row is not None:
                board_copy[row][col] = self.player_id
                if self._winning_move(board_copy, self.player_id, game.rows, game.columns):
                    return col

        # 2. Check for a move that blocks the opponent's win.
        opponent = 2 if self.player_id == 1 else 1
        for col in valid_columns:
            board_copy = [row[:] for row in game.board]
            row = self._get_next_open_row(board_copy, col)
            if row is not None:
                board_copy[row][col] = opponent
                if self._winning_move(board_copy, opponent, game.rows, game.columns):
                    return col

        # 3. Prefer the center column if it's available.
        center = game.columns // 2
        if center in valid_columns:
            return center

        # 4. Otherwise, choose the column that is closest to the center.
        best_score = -float('inf')
        best_col = None
        for col in valid_columns:
            # A simple scoring: the closer to center, the higher the score.
            score = -abs(col - center)
            if score > best_score:
                best_score = score
                best_col = col

        # Fallback in case best_col is None (should not happen if valid_columns is not empty)
        return best_col if best_col is not None else random.choice(valid_columns)


import random
import time
import datetime

class MinimaxBot(Bot):
    def __init__(self, player_id, max_depth=None):
        """
        Initialize the MinimaxBot.
        - player_id: The player's id (1 or 2).
        - max_depth: Optional fixed maximum depth. If None, iterative deepening runs until time is up.
        """
        super().__init__(player_id)
        self.max_depth = max_depth

    def choose_move(self, game):
        """
        Uses iterative deepening minimax search with alpha-beta pruning to choose a move.
        The bot has a 2-second time limit per move.
        Returns a valid column number.
        """
        start_time = time.time()
        time_limit = 2.0  # seconds
        valid_locations = [col for col in range(game.columns) if game.is_valid_move(col)]
        if not valid_locations:
            raise Exception("No valid moves available!")
        best_move = random.choice(valid_locations)  # Fallback in case time runs out quickly
        current_depth = 1

        try:
            # Iterative deepening loop
            while True:
                # If a max_depth is set and we've reached it, break.
                if self.max_depth is not None and current_depth > self.max_depth:
                    break

                score, column = self.minimax(game.board, current_depth, -float('inf'), float('inf'),
                                               True, start_time, time_limit, game)
                # Only update if we got a valid column and we're still within time.
                if time.time() - start_time < time_limit and column is not None:
                    best_move = column

                current_depth += 1
        except TimeoutError:
            # Time limit exceeded; use the best move from the last completed iteration.
            pass

        return best_move

    def minimax(self, board, depth, alpha, beta, maximizingPlayer, start_time, time_limit, game):
        """
        Recursive minimax function with alpha-beta pruning.
        If the time limit is exceeded, a TimeoutError is raised.
        Returns a tuple (score, column).
        """
        # Check the time limit
        if time.time() - start_time >= time_limit:
            raise TimeoutError

        valid_locations = self.get_valid_locations(board, game)
        is_terminal = self.is_terminal_node(board, game)
        if depth == 0 or is_terminal:
            if is_terminal:
                if self.winning_move(board, self.player_id, game):
                    return (float('inf'), None)
                elif self.winning_move(board, 2 if self.player_id == 1 else 1, game):
                    return (-float('inf'), None)
                else:
                    return (0, None)  # Game is over (draw)
            else:
                return (self.score_position(board, self.player_id, game), None)

        if maximizingPlayer:
            value = -float('inf')
            best_column = random.choice(valid_locations)
            for col in valid_locations:
                row = self.get_next_open_row(board, col, game.rows)
                if row is not None:
                    board_copy = [r[:] for r in board]  # Deep copy the board
                    board_copy[row][col] = self.player_id
                    new_score, _ = self.minimax(board_copy, depth - 1, alpha, beta,
                                                False, start_time, time_limit, game)
                    if new_score > value:
                        value = new_score
                        best_column = col
                    alpha = max(alpha, value)
                    if alpha >= beta:
                        break
            return value, best_column
        else:
            value = float('inf')
            best_column = random.choice(valid_locations)
            opponent = 2 if self.player_id == 1 else 1
            for col in valid_locations:
                row = self.get_next_open_row(board, col, game.rows)
                if row is not None:
                    board_copy = [r[:] for r in board]
                    board_copy[row][col] = opponent
                    new_score, _ = self.minimax(board_copy, depth - 1, alpha, beta,
                                                True, start_time, time_limit, game)
                    if new_score < value:
                        value = new_score
                        best_column = col
                    beta = min(beta, value)
                    if alpha >= beta:
                        break
            return value, best_column

    def get_valid_locations(self, board, game):
        """Return a list of column indices where a move is valid."""
        valid_locations = []
        for col in range(game.columns):
            if board[0][col] == 0:
                valid_locations.append(col)
        return valid_locations

    def get_next_open_row(self, board, col, rows):
        """Return the next open row in the given column."""
        for r in range(rows - 1, -1, -1):
            if board[r][col] == 0:
                return r
        return None

    def winning_move(self, board, piece, game):
        """Check whether the given board has a winning move for the specified piece."""
        # Check horizontal locations for win
        for r in range(game.rows):
            for c in range(game.columns - 3):
                if board[r][c] == piece and board[r][c+1] == piece and \
                   board[r][c+2] == piece and board[r][c+3] == piece:
                    return True

        # Check vertical locations for win
        for r in range(game.rows - 3):
            for c in range(game.columns):
                if board[r][c] == piece and board[r+1][c] == piece and \
                   board[r+2][c] == piece and board[r+3][c] == piece:
                    return True

        # Check positively sloped diagonals
        for r in range(game.rows - 3):
            for c in range(game.columns - 3):
                if board[r][c] == piece and board[r+1][c+1] == piece and \
                   board[r+2][c+2] == piece and board[r+3][c+3] == piece:
                    return True

        # Check negatively sloped diagonals
        for r in range(3, game.rows):
            for c in range(game.columns - 3):
                if board[r][c] == piece and board[r-1][c+1] == piece and \
                   board[r-2][c+2] == piece and board[r-3][c+3] == piece:
                    return True

        return False

    def is_terminal_node(self, board, game):
        """Returns True if the board state is terminal (win or draw)."""
        return (self.winning_move(board, self.player_id, game) or
                self.winning_move(board, 2 if self.player_id == 1 else 1, game) or
                len(self.get_valid_locations(board, game)) == 0)

    def score_position(self, board, piece, game):
        """
        Evaluate the board and return a score from the perspective of the given piece.
        A simple heuristic: center control plus counting potential windows.
        """
        score = 0
        center = game.columns // 2
        center_array = [board[r][center] for r in range(game.rows)]
        center_count = center_array.count(piece)
        score += center_count * 3

        # Score Horizontal
        for r in range(game.rows):
            row_array = board[r]
            for c in range(game.columns - 3):
                window = row_array[c:c+4]
                score += self.evaluate_window(window, piece)

        # Score Vertical
        for c in range(game.columns):
            col_array = [board[r][c] for r in range(game.rows)]
            for r in range(game.rows - 3):
                window = col_array[r:r+4]
                score += self.evaluate_window(window, piece)

        # Score positively sloped diagonal
        for r in range(game.rows - 3):
            for c in range(game.columns - 3):
                window = [board[r+i][c+i] for i in range(4)]
                score += self.evaluate_window(window, piece)

        # Score negatively sloped diagonal
        for r in range(3, game.rows):
            for c in range(game.columns - 3):
                window = [board[r-i][c+i] for i in range(4)]
                score += self.evaluate_window(window, piece)

        return score

    def evaluate_window(self, window, piece):
        """
        Evaluate a window of four cells and return a score.
        +100 for 4 in a row, +5 for 3 with an empty, +2 for 2 with two empties.
        Subtract if the opponent is close to winning.
        """
        score = 0
        opponent = 2 if piece == 1 else 1

        if window.count(piece) == 4:
            score += 100
        elif window.count(piece) == 3 and window.count(0) == 1:
            score += 5
        elif window.count(piece) == 2 and window.count(0) == 2:
            score += 2

        if window.count(opponent) == 3 and window.count(0) == 1:
            score -= 4

        return score


import random
import time
import datetime
import copy

class MCTSBot(Bot):
    def choose_move(self, game):
        """
        Uses Monte Carlo Tree Search (MCTS) to select a move.
        Runs simulations for up to 2 seconds and then returns the move (column)
        with the highest win ratio.
        """
        time_limit = 2.0  # seconds per move
        start_time = time.time()

        # Get list of valid moves from current board.
        valid_moves = self.get_valid_moves(game.board, game)
        if not valid_moves:
            raise Exception("No valid moves available!")

        # Initialize simulation results for each valid move:
        # Each key maps to a tuple: [wins, simulations]
        results = {col: [0, 0] for col in valid_moves}

        # Run simulations until time limit is reached.
        while time.time() - start_time < time_limit:
            for col in valid_moves:
                # Check time at each simulation iteration.
                if time.time() - start_time >= time_limit:
                    break
                # Copy the current board so we don't affect the real game.
                board_copy = copy.deepcopy(game.board)
                row = self.get_next_open_row(board_copy, col, game.rows)
                if row is None:
                    continue  # Should not happen as move is valid.
                board_copy[row][col] = self.player_id
                # Set the next player for the simulation.
                current_player = 2 if self.player_id == 1 else 1
                # Run a random simulation (playout) from this board state.
                outcome = self.simulate_random_game(board_copy, current_player, game)
                # Outcome is from the perspective of our bot:
                # 1 for win, 0.5 for draw, 0 for loss.
                results[col][0] += outcome
                results[col][1] += 1

        # Choose the move with the best win ratio.
        best_move = None
        best_ratio = -float('inf')
        for col in results:
            wins, simulations = results[col]
            if simulations > 0:
                ratio = wins / simulations
                # Debug print for simulation results:
                # print(f"Col {col}: {wins}/{simulations} = {ratio:.3f}")
                if ratio > best_ratio:
                    best_ratio = ratio
                    best_move = col

        # Fallback: if no simulations were run (should not happen), choose randomly.
        if best_move is None:
            best_move = random.choice(valid_moves)
        return best_move

    def simulate_random_game(self, board, current_player, game):
        """
        Simulate a random playout (i.e. play random moves until terminal state).
        Returns:
            1 if the outcome is a win for self.player_id,
            0.5 if the outcome is a draw,
            0 if the outcome is a loss.
        """
        # Continue simulation until a terminal state is reached.
        while True:
            valid_moves = self.get_valid_moves(board, game)
            if not valid_moves:
                # No more moves: board full, treat as draw.
                return 0.5
            # Choose a random move.
            col = random.choice(valid_moves)
            row = self.get_next_open_row(board, col, game.rows)
            if row is None:
                continue
            board[row][col] = current_player
            # Check if this move wins the game.
            if self.winning_move(board, current_player, game):
                # Return outcome from the perspective of our bot.
                return 1 if current_player == self.player_id else 0
            # Check for draw: if board is full.
            if all(board[0][c] != 0 for c in range(game.columns)):
                return 0.5
            # Switch players.
            current_player = 2 if current_player == 1 else 1

    def get_valid_moves(self, board, game):
        """Return a list of valid column indices where a move can be made."""
        valid_moves = []
        for col in range(game.columns):
            if board[0][col] == 0:
                valid_moves.append(col)
        return valid_moves

    def get_next_open_row(self, board, col, rows):
        """Return the next open row in the specified column."""
        for r in range(rows - 1, -1, -1):
            if board[r][col] == 0:
                return r
        return None

    def winning_move(self, board, piece, game):
        """
        Check whether the given board has a winning move for the specified piece.
        This function mirrors the winning_move logic from our Connect4Game.
        """
        # Check horizontal locations for win.
        for r in range(game.rows):
            for c in range(game.columns - 3):
                if (board[r][c] == piece and board[r][c+1] == piece and
                    board[r][c+2] == piece and board[r][c+3] == piece):
                    return True

        # Check vertical locations for win.
        for r in range(game.rows - 3):
            for c in range(game.columns):
                if (board[r][c] == piece and board[r+1][c] == piece and
                    board[r+2][c] == piece and board[r+3][c] == piece):
                    return True

        # Check positively sloped diagonals.
        for r in range(game.rows - 3):
            for c in range(game.columns - 3):
                if (board[r][c] == piece and board[r+1][c+1] == piece and
                    board[r+2][c+2] == piece and board[r+3][c+3] == piece):
                    return True

        # Check negatively sloped diagonals.
        for r in range(3, game.rows):
            for c in range(game.columns - 3):
                if (board[r][c] == piece and board[r-1][c+1] == piece and
                    board[r-2][c+2] == piece and board[r-3][c+3] == piece):
                    return True

        return False




# --- Assume all the previous classes (Connect4Game, Bot, RandomBot, HeuristicBot, MinimaxBot, MCTSBot) are defined above ---
# For this simulation script, we assume they are available in the same module or imported accordingly.

# If you have them in separate files, you can import them like:
# from connect4 import Connect4Game
# from bots import RandomBot, HeuristicBot, MinimaxBot, MCTSBot


def run_game(bot1_class, bot2_class, bot1_name, bot2_name, game_number, ordering):
    """
    Runs a single game between two bot classes.

    Parameters:
        bot1_class, bot2_class: The classes for the two bots.
        bot1_name, bot2_name: Their string identifiers.
        game_number: An integer for the game number.
        ordering: "normal" means bot1 is Player 1 and bot2 is Player 2;
                  "reversed" means bot2 is Player 1 and bot1 is Player 2.

    Returns:
        A dictionary containing metadata and the move log of the game.
    """
    game = Connect4Game()  # Create a new Connect 4 game instance.

    # Instantiate the bots according to the ordering.
    if ordering == "normal":
        player1 = bot1_class(player_id=1)
        player2 = bot2_class(player_id=2)
        bot1_id = bot1_name
        bot2_id = bot2_name
    else:  # "reversed" ordering.
        player1 = bot2_class(player_id=1)
        player2 = bot1_class(player_id=2)
        bot1_id = bot2_name  # In the metadata, bot1 is always the one playing as Player 1.
        bot2_id = bot1_name

    # Game loop: Continue until the game ends.
    while not game.game_over:
        if game.current_player == 1:
            chosen_move = player1.choose_move(game)
        else:
            chosen_move = player2.choose_move(game)

        # Make the move (our Connect4Game.make_move handles validity and logging).
        valid = game.make_move(chosen_move)
        if not valid:
            # This is a fallback: if a bot somehow returns an invalid move, pick the first valid move.
            valid_moves = [col for col in range(game.columns) if game.is_valid_move(col)]
            if valid_moves:
                game.make_move(valid_moves[0])
            else:
                break  # No valid moves remain.

        # Safety check: stop if the board is full.
        if len(game.move_log) >= game.rows * game.columns:
            break

    # Determine the winner.
    # We assume that when a winning move is made, game.game_over is set.
    if game.move_log:
        # The last move recorded is assumed to be the winning move.
        winning_player = game.move_log[-1]["player"]
        # Map the winning player to the corresponding bot ID.
        if ordering == "normal":
            winner_bot = bot1_id if winning_player == 1 else bot2_id
        else:
            winner_bot = bot1_id if winning_player == 1 else bot2_id
    else:
        winner_bot = "Draw"

    # Build the game record with metadata.
    game_record = {
        "metadata": {
            "bot1": bot1_id,
            "bot2": bot2_id,
            "game_number": game_number,
            "ordering": ordering,  # Indicates which bot was assigned Player 1.
            "result": winner_bot
        },
        "move_log": game.move_log
    }
    return game_record


def run_simulation():
    """
    Runs a tournament simulation where every pair of distinct bots plays 100 games.
    For each pairing:
      - 50 games are played with one bot as Player 1.
      - 50 games are played with the roles reversed.
    Each game is saved as a JSON file in the 'simulation_game_logs' directory.
    """
    # Create the directory to store game logs.
    simulation_dir = "simulation_game_logs"
    os.makedirs(simulation_dir, exist_ok=True)

    # Mapping of bot names to their classes.
    bot_classes = {
        "RandomBot": RandomBot,
        "HeuristicBot": HeuristicBot,
        "MinimaxBot": MinimaxBot,
        "MCTSBot": MCTSBot
    }
    bot_names = list(bot_classes.keys())
    total_games = 0

    # Iterate over each distinct pairing (do not pair a bot with itself).
    for i in range(len(bot_names)):
        for j in range(i + 1, len(bot_names)):
            bot1_name = bot_names[i]
            bot2_name = bot_names[j]
            bot1_class = bot_classes[bot1_name]
            bot2_class = bot_classes[bot2_name]

            # Run 50 games with "normal" ordering: bot1 is Player 1.
            for game_index in range(1, 51):
                record = run_game(bot1_class, bot2_class, bot1_name, bot2_name,
                                  game_number=game_index, ordering="normal")
                filename = os.path.join(simulation_dir,
                                        f"{bot1_name}_vs_{bot2_name}_normal_game{game_index}.json")
                with open(filename, "w") as f:
                    json.dump(record, f, indent=4)
                total_games += 1

            # Run 50 games with "reversed" ordering: bot2 is Player 1.
            for game_index in range(51, 101):
                record = run_game(bot1_class, bot2_class, bot1_name, bot2_name,
                                  game_number=game_index, ordering="reversed")
                filename = os.path.join(simulation_dir,
                                        f"{bot1_name}_vs_{bot2_name}_reversed_game{game_index}.json")
                with open(filename, "w") as f:
                    json.dump(record, f, indent=4)
                total_games += 1

    print(f"Simulation completed. Total games played: {total_games}")


if __name__ == "__main__":
    run_simulation()


Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 w

In [1]:
import os
import json
import random
import time
import datetime
import copy



class Connect4Game:
    def __init__(self, rows=6, columns=7):
        """
        Initialize the Connect 4 board and game state.
        - rows: Number of rows (default 6).
        - columns: Number of columns (default 7).
        """
        self.rows = rows
        self.columns = columns
        # Create a rows x columns board initialized with zeros (empty cells)
        self.board = [[0 for _ in range(columns)] for _ in range(rows)]
        # List to store move notation for analysis
        self.move_log = []
        # Player 1 starts (we use 1 and 2 to represent players)
        self.current_player = 1
        # Game state
        self.game_over = False

    def is_valid_move(self, column):
        """Return True if the top cell in the column is empty (i.e., the move is valid)."""
        return self.board[0][column] == 0

    def get_next_open_row(self, column):
        """
        Return the next open row in the specified column.
        We fill from the bottom of the board upward.
        """
        for r in range(self.rows-1, -1, -1):
            if self.board[r][column] == 0:
                return r
        return None

    def drop_piece(self, row, column, piece):
        """Place a piece (1 or 2) in the board at the specified row and column."""
        self.board[row][column] = piece

    def make_move(self, column):
        """
        Attempt to make a move in the specified column.
        Returns True if move is successful; False if the column is full or invalid.
        Records the move in the move_log.
        """
        if column < 0 or column >= self.columns:
            raise ValueError("Column index out of range.")

        if not self.is_valid_move(column):
            print(f"Column {column} is full. Move invalid.")
            return False

        row = self.get_next_open_row(column)
        if row is None:
            print(f"No open row found in column {column}.")
            return False

        # Drop the piece on the board.
        self.drop_piece(row, column, self.current_player)

        # Record move notation
        move_entry = {
            "move_number": len(self.move_log) + 1,
            "player": self.current_player,
            "column": column,
            "row": row,
            "timestamp": datetime.datetime.now().isoformat()
        }
        self.move_log.append(move_entry)

        # Check for a win after the move.
        if self.winning_move(self.current_player):
            self.game_over = True
            print(f"Player {self.current_player} wins!")

        # Switch player
        self.current_player = 2 if self.current_player == 1 else 1

        return True

    def winning_move(self, piece):
        """Check whether the current board has a winning move for the given piece."""
        # Check horizontal locations for win
        for c in range(self.columns - 3):
            for r in range(self.rows):
                if (self.board[r][c] == piece and self.board[r][c+1] == piece and
                    self.board[r][c+2] == piece and self.board[r][c+3] == piece):
                    return True

        # Check vertical locations for win
        for c in range(self.columns):
            for r in range(self.rows - 3):
                if (self.board[r][c] == piece and self.board[r+1][c] == piece and
                    self.board[r+2][c] == piece and self.board[r+3][c] == piece):
                    return True

        # Check positively sloped diagonals
        for c in range(self.columns - 3):
            for r in range(self.rows - 3):
                if (self.board[r][c] == piece and self.board[r+1][c+1] == piece and
                    self.board[r+2][c+2] == piece and self.board[r+3][c+3] == piece):
                    return True

        # Check negatively sloped diagonals
        for c in range(self.columns - 3):
            for r in range(3, self.rows):
                if (self.board[r][c] == piece and self.board[r-1][c+1] == piece and
                    self.board[r-2][c+2] == piece and self.board[r-3][c+3] == piece):
                    return True

        return False

    def print_board(self):
        """Print the current board state. (Row 0 is the top of the board)"""
        for row in self.board:
            print(row)
        print("\n")

    def save_game_log(self, filename="connect4_game_log.json"):
        """
        Save the recorded move log to a JSON file.
        This file can later be used for analysis.
        """
        with open(filename, "w") as f:
            json.dump(self.move_log, f, indent=4)
        print(f"Game log saved to {filename}")


class Bot:
    def __init__(self, player_id):
        """
        Initialize the bot with its player id (1 or 2).
        """
        self.player_id = player_id

    def choose_move(self, game):
        """
        Given the current game state, return a valid column number.
        This method should be overridden by specific bot implementations.
        """
        raise NotImplementedError("This method should be implemented by subclasses.")

    def format_move(self, game, column):
        """
        Given the chosen column and game state, determine the row where the piece will land.
        Return a move dictionary in the standardized format.
        """
        row = game.get_next_open_row(column)
        move_entry = {
            "move_number": len(game.move_log) + 1,
            "player": self.player_id,
            "column": column,
            "row": row,
            "timestamp": datetime.datetime.now().isoformat()
        }
        return move_entry

class RandomBot(Bot):
    def choose_move(self, game):
        """
        Chooses a random valid move from the available columns.
        Returns:
            An integer representing the chosen column (0-indexed).
        """
        # Get a list of valid columns (where the top cell is still empty)
        valid_columns = [col for col in range(game.columns) if game.is_valid_move(col)]

        if not valid_columns:
            raise Exception("No valid moves available!")

        # Randomly choose one of the valid columns.
        chosen_column = random.choice(valid_columns)
        return chosen_column


class HeuristicBot(Bot):
    def _get_next_open_row(self, board, col):
        """
        Given a board state (a 2D list) and a column,
        return the next open row (from bottom) in that column.
        """
        for r in range(len(board) - 1, -1, -1):
            if board[r][col] == 0:
                return r
        return None

    def _winning_move(self, board, piece, rows, columns):
        """
        Check if placing a piece on the given board results in a win.
        This function mirrors the winning_move checks in the game class.
        """
        # Check horizontal locations for win
        for r in range(rows):
            for c in range(columns - 3):
                if (board[r][c] == piece and board[r][c+1] == piece and
                    board[r][c+2] == piece and board[r][c+3] == piece):
                    return True

        # Check vertical locations for win
        for r in range(rows - 3):
            for c in range(columns):
                if (board[r][c] == piece and board[r+1][c] == piece and
                    board[r+2][c] == piece and board[r+3][c] == piece):
                    return True

        # Check positively sloped diagonals
        for r in range(rows - 3):
            for c in range(columns - 3):
                if (board[r][c] == piece and board[r+1][c+1] == piece and
                    board[r+2][c+2] == piece and board[r+3][c+3] == piece):
                    return True

        # Check negatively sloped diagonals
        for r in range(3, rows):
            for c in range(columns - 3):
                if (board[r][c] == piece and board[r-1][c+1] == piece and
                    board[r-2][c+2] == piece and board[r-3][c+3] == piece):
                    return True

        return False

    def choose_move(self, game):
        """
        Implements the heuristic decision-making:
        1. Check for an immediate winning move.
        2. Check for an immediate move to block the opponent.
        3. Prefer the center column if available.
        4. Otherwise, choose a column closest to the center.
        Returns a valid column number.
        """
        valid_columns = [col for col in range(game.columns) if game.is_valid_move(col)]

        # 1. Check for an immediate winning move.
        for col in valid_columns:
            board_copy = [row[:] for row in game.board]  # deep copy of the board
            row = self._get_next_open_row(board_copy, col)
            if row is not None:
                board_copy[row][col] = self.player_id
                if self._winning_move(board_copy, self.player_id, game.rows, game.columns):
                    return col

        # 2. Check for a move that blocks the opponent's win.
        opponent = 2 if self.player_id == 1 else 1
        for col in valid_columns:
            board_copy = [row[:] for row in game.board]
            row = self._get_next_open_row(board_copy, col)
            if row is not None:
                board_copy[row][col] = opponent
                if self._winning_move(board_copy, opponent, game.rows, game.columns):
                    return col

        # 3. Prefer the center column if it's available.
        center = game.columns // 2
        if center in valid_columns:
            return center

        # 4. Otherwise, choose the column that is closest to the center.
        best_score = -float('inf')
        best_col = None
        for col in valid_columns:
            # A simple scoring: the closer to center, the higher the score.
            score = -abs(col - center)
            if score > best_score:
                best_score = score
                best_col = col

        # Fallback in case best_col is None (should not happen if valid_columns is not empty)
        return best_col if best_col is not None else random.choice(valid_columns)



import random
import time
import datetime
import copy

class MinimaxBot(Bot):
    def __init__(self, player_id, max_depth=None):
        """
        Initialize the MinimaxBot.
        - player_id: The player's id (1 or 2).
        - max_depth: Optional fixed maximum depth. If None, iterative deepening runs until time is up.
        """
        super().__init__(player_id)
        self.max_depth = max_depth
        self.move_memory = {}  # Dictionary to store move evaluations

    def choose_move(self, game):
        """
        Uses iterative deepening minimax search with alpha-beta pruning to choose a move.
        The bot has a 2-second time limit per move.
        Returns a valid column number.
        """
        start_time = time.time()
        time_limit = 2.0  # seconds
        valid_locations = [col for col in range(game.columns) if game.is_valid_move(col)]
        if not valid_locations:
            raise Exception("No valid moves available!")
        best_move = random.choice(valid_locations)  # Fallback in case time runs out quickly
        current_depth = 1

        try:
            # Iterative deepening loop
            while True:
                # If a max_depth is set and we've reached it, break.
                if self.max_depth is not None and current_depth > self.max_depth:
                    break

                score, column = self.minimax(game.board, current_depth, -float('inf'), float('inf'),
                                               True, start_time, time_limit, game)
                # Only update if we got a valid column and we're still within time.
                if time.time() - start_time < time_limit and column is not None:
                    best_move = column

                current_depth += 1
        except TimeoutError:
            # Time limit exceeded; use the best move from the last completed iteration.
            pass

        return best_move

    def minimax(self, board, depth, alpha, beta, maximizingPlayer, start_time, time_limit, game):
        """
        Recursive minimax function with alpha-beta pruning.
        If the time limit is exceeded, a TimeoutError is raised.
        Returns a tuple (score, column).
        """
        # Check the time limit
        if time.time() - start_time >= time_limit:
            raise TimeoutError

        # Convert the board to a tuple so it can be used as a dictionary key
        board_tuple = tuple(tuple(row) for row in board)

        valid_locations = self.get_valid_locations(board, game)
        is_terminal = self.is_terminal_node(board, game)

        if depth == 0 or is_terminal:
            if is_terminal:
                if self.winning_move(board, self.player_id, game):
                    return (float('inf'), None)
                elif self.winning_move(board, 2 if self.player_id == 1 else 1, game):
                    return (-float('inf'), None)
                else:
                    return (0, None)  # Game is over (draw)
            else:
                return (self.score_position(board, self.player_id, game), None)

        if maximizingPlayer:
            value = -float('inf')
            best_column = random.choice(valid_locations)

            # Prioritize moves in move_memory
            sorted_moves = sorted(valid_locations, key=lambda col: self.move_memory.get((board_tuple, col), 0), reverse=True)

            for col in sorted_moves:
                row = self.get_next_open_row(board, col, game.rows)
                if row is not None:
                    board_copy = [r[:] for r in board]  # Deep copy the board
                    board_copy[row][col] = self.player_id
                    new_score, _ = self.minimax(board_copy, depth - 1, alpha, beta,
                                                False, start_time, time_limit, game)

                    # Update move_memory after evaluating move
                    if new_score > value:
                        value = new_score
                        best_column = col
                        self.move_memory[(board_tuple, col)] = new_score # store the evaluation for the board and column
                    else:
                        self.move_memory[(board_tuple, col)] = -1 # penalize bad moves

                    alpha = max(alpha, value)
                    if alpha >= beta:
                        break
            return value, best_column
        else:
            value = float('inf')
            best_column = random.choice(valid_locations)

             # Prioritize moves in move_memory
            sorted_moves = sorted(valid_locations, key=lambda col: self.move_memory.get((board_tuple, col), 0)) #sort the move_memory in ascending order

            opponent = 2 if self.player_id == 1 else 1
            for col in sorted_moves:
                row = self.get_next_open_row(board, col, game.rows)
                if row is not None:
                    board_copy = [r[:] for r in board]
                    board_copy[row][col] = opponent
                    new_score, _ = self.minimax(board_copy, depth - 1, alpha, beta,
                                                True, start_time, time_limit, game)

                    # Update move_memory after evaluating move
                    if new_score < value:
                        value = new_score
                        best_column = col
                        self.move_memory[(board_tuple, col)] = new_score # store the evaluation for the board and column
                    else:
                        self.move_memory[(board_tuple, col)] = 1 #penalize good moves for opponent to avoid

                    beta = min(beta, value)
                    if alpha >= beta:
                        break
            return value, best_column

    def get_valid_locations(self, board, game):
        """Return a list of column indices where a move is valid."""
        valid_locations = []
        for col in range(game.columns):
            if board[0][col] == 0:
                valid_locations.append(col)
        return valid_locations

    def get_next_open_row(self, board, col, rows):
        """Return the next open row in the given column."""
        for r in range(rows - 1, -1, -1):
            if board[r][col] == 0:
                return r
        return None

    def winning_move(self, board, piece, game):
        """Check whether the given board has a winning move for the specified piece."""
        # Check horizontal locations for win
        for r in range(game.rows):
            for c in range(game.columns - 3):
                if board[r][c] == piece and board[r][c+1] == piece and \
                   board[r][c+2] == piece and board[r][c+3] == piece:
                    return True

        # Check vertical locations for win
        for r in range(game.rows - 3):
            for c in range(game.columns):
                if board[r][c] == piece and board[r+1][c] == piece and \
                   board[r+2][c] == piece and board[r+3][c] == piece:
                    return True

        # Check positively sloped diagonals
        for r in range(game.rows - 3):
            for c in range(game.columns - 3):
                if board[r][c] == piece and board[r+1][c+1] == piece and \
                   board[r+2][c+2] == piece and board[r+3][c+3] == piece:
                    return True

        # Check negatively sloped diagonals
        for r in range(3, game.rows):
            for c in range(game.columns - 3):
                if board[r][c] == piece and board[r-1][c+1] == piece and \
                   board[r-2][c+2] == piece and board[r-3][c+3] == piece:
                    return True

        return False

    def is_terminal_node(self, board, game):
        """Returns True if the board state is terminal (win or draw)."""
        return (self.winning_move(board, self.player_id, game) or
                self.winning_move(board, 2 if self.player_id == 1 else 1, game) or
                len(self.get_valid_locations(board, game)) == 0)

    def score_position(self, board, piece, game):
        """
        Evaluate the board and return a score from the perspective of the given piece.
        A simple heuristic: center control plus counting potential windows.
        """
        score = 0
        center = game.columns // 2
        center_array = [board[r][center] for r in range(game.rows)]
        center_count = center_array.count(piece)
        score += center_count * 3

        # Score Horizontal
        for r in range(game.rows):
            row_array = board[r]
            for c in range(game.columns - 3):
                window = row_array[c:c+4]
                score += self.evaluate_window(window, piece)

        # Score Vertical
        for c in range(game.columns):
            col_array = [board[r][c] for r in range(game.rows)]
            for r in range(game.rows - 3):
                window = col_array[r:r+4]
                score += self.evaluate_window(window, piece)

        # Score positively sloped diagonal
        for r in range(game.rows - 3):
            for c in range(game.columns - 3):
                window = [board[r+i][c+i] for i in range(4)]
                score += self.evaluate_window(window, piece)

        # Score negatively sloped diagonal
        for r in range(3, game.rows):
            for c in range(game.columns - 3):
                window = [board[r-i][c+i] for i in range(4)]
                score += self.evaluate_window(window, piece)

        return score

    def evaluate_window(self, window, piece):
        """
        Evaluate a window of four cells and return a score.
        +100 for 4 in a row, +5 for 3 with an empty, +2 for 2 with two empties.
        Subtract if the opponent is close to winning.
        """
        score = 0
        opponent = 2 if piece == 1 else 1

        if window.count(piece) == 4:
            score += 100
        elif window.count(piece) == 3 and window.count(0) == 1:
            score += 5
        elif window.count(piece) == 2 and window.count(0) == 2:
            score += 2

        if window.count(opponent) == 3 and window.count(0) == 1:
            score -= 4

        return score

import random
import time
import datetime
import copy

class MCTSBot(Bot):
    def choose_move(self, game):
        """
        Uses Monte Carlo Tree Search (MCTS) to select a move.
        Runs simulations for up to 2 seconds and then returns the move (column)
        with the highest win ratio.
        """
        time_limit = 2.0  # seconds per move
        start_time = time.time()

        # Get list of valid moves from current board.
        valid_moves = self.get_valid_moves(game.board, game)
        if not valid_moves:
            raise Exception("No valid moves available!")

        # Initialize simulation results for each valid move:
        # Each key maps to a tuple: [wins, simulations]
        results = {col: [0, 0] for col in valid_moves}

        # Run simulations until time limit is reached.
        while time.time() - start_time < time_limit:
            for col in valid_moves:
                # Check time at each simulation iteration.
                if time.time() - start_time >= time_limit:
                    break
                # Copy the current board so we don't affect the real game.
                board_copy = copy.deepcopy(game.board)
                row = self.get_next_open_row(board_copy, col, game.rows)
                if row is None:
                    continue  # Should not happen as move is valid.
                board_copy[row][col] = self.player_id
                # Set the next player for the simulation.
                current_player = 2 if self.player_id == 1 else 1
                # Run a random simulation (playout) from this board state.
                outcome = self.simulate_random_game(board_copy, current_player, game)
                # Outcome is from the perspective of our bot:
                # 1 for win, 0.5 for draw, 0 for loss.
                results[col][0] += outcome
                results[col][1] += 1

        # Choose the move with the best win ratio.
        best_move = None
        best_ratio = -float('inf')
        for col in results:
            wins, simulations = results[col]
            if simulations > 0:
                ratio = wins / simulations
                # Debug print for simulation results:
                # print(f"Col {col}: {wins}/{simulations} = {ratio:.3f}")
                if ratio > best_ratio:
                    best_ratio = ratio
                    best_move = col

        # Fallback: if no simulations were run (should not happen), choose randomly.
        if best_move is None:
            best_move = random.choice(valid_moves)
        return best_move

    def simulate_random_game(self, board, current_player, game):
        """
        Simulate a random playout (i.e. play random moves until terminal state).
        Returns:
            1 if the outcome is a win for self.player_id,
            0.5 if the outcome is a draw,
            0 if the outcome is a loss.
        """
        # Continue simulation until a terminal state is reached.
        while True:
            valid_moves = self.get_valid_moves(board, game)
            if not valid_moves:
                # No more moves: board full, treat as draw.
                return 0.5
            # Choose a random move.
            col = random.choice(valid_moves)
            row = self.get_next_open_row(board, col, game.rows)
            if row is None:
                continue
            board[row][col] = current_player
            # Check if this move wins the game.
            if self.winning_move(board, current_player, game):
                # Return outcome from the perspective of our bot.
                return 1 if current_player == self.player_id else 0
            # Check for draw: if board is full.
            if all(board[0][c] != 0 for c in range(game.columns)):
                return 0.5
            # Switch players.
            current_player = 2 if current_player == 1 else 1

    def get_valid_moves(self, board, game):
        """Return a list of valid column indices where a move can be made."""
        valid_moves = []
        for col in range(game.columns):
            if board[0][col] == 0:
                valid_moves.append(col)
        return valid_moves

    def get_next_open_row(self, board, col, rows):
        """Return the next open row in the specified column."""
        for r in range(rows - 1, -1, -1):
            if board[r][col] == 0:
                return r
        return None

    def winning_move(self, board, piece, game):
        """
        Check whether the given board has a winning move for the specified piece.
        This function mirrors the winning_move logic from our Connect4Game.
        """
        # Check horizontal locations for win.
        for r in range(game.rows):
            for c in range(game.columns - 3):
                if (board[r][c] == piece and board[r][c+1] == piece and
                    board[r][c+2] == piece and board[r][c+3] == piece):
                    return True

        # Check vertical locations for win.
        for r in range(game.rows - 3):
            for c in range(game.columns):
                if (board[r][c] == piece and board[r+1][c] == piece and
                    board[r+2][c] == piece and board[r+3][c] == piece):
                    return True

        # Check positively sloped diagonals.
        for r in range(game.rows - 3):
            for c in range(game.columns - 3):
                if (board[r][c] == piece and board[r+1][c+1] == piece and
                    board[r+2][c+2] == piece and board[r+3][c+3] == piece):
                    return True

        # Check negatively sloped diagonals.
        for r in range(3, game.rows):
            for c in range(game.columns - 3):
                if (board[r][c] == piece and board[r-1][c+1] == piece and
                    board[r-2][c+2] == piece and board[r-3][c+3] == piece):
                    return True

        return False




# --- Assume all the previous classes (Connect4Game, Bot, RandomBot, HeuristicBot, MinimaxBot, MCTSBot) are defined above ---
# For this simulation script, we assume they are available in the same module or imported accordingly.

# If you have them in separate files, you can import them like:
# from connect4 import Connect4Game
# from bots import RandomBot, HeuristicBot, MinimaxBot, MCTSBot


def run_game(bot1_class, bot2_class, bot1_name, bot2_name, game_number, ordering):
    """
    Runs a single game between two bot classes.

    Parameters:
        bot1_class, bot2_class: The classes for the two bots.
        bot1_name, bot2_name: Their string identifiers.
        game_number: An integer for the game number.
        ordering: "normal" means bot1 is Player 1 and bot2 is Player 2;
                  "reversed" means bot2 is Player 1 and bot1 is Player 2.

    Returns:
        A dictionary containing metadata and the move log of the game.
    """
    game = Connect4Game()  # Create a new Connect 4 game instance.

    # Instantiate the bots according to the ordering.
    if ordering == "normal":
        player1 = bot1_class(player_id=1)
        player2 = bot2_class(player_id=2)
        bot1_id = bot1_name
        bot2_id = bot2_name
    else:  # "reversed" ordering.
        player1 = bot2_class(player_id=1)
        player2 = bot1_class(player_id=2)
        bot1_id = bot2_name  # In the metadata, bot1 is always the one playing as Player 1.
        bot2_id = bot1_name

    # Game loop: Continue until the game ends.
    while not game.game_over:
        if game.current_player == 1:
            chosen_move = player1.choose_move(game)
        else:
            chosen_move = player2.choose_move(game)

        # Make the move (our Connect4Game.make_move handles validity and logging).
        valid = game.make_move(chosen_move)
        if not valid:
            # This is a fallback: if a bot somehow returns an invalid move, pick the first valid move.
            valid_moves = [col for col in range(game.columns) if game.is_valid_move(col)]
            if valid_moves:
                game.make_move(valid_moves[0])
            else:
                break  # No valid moves remain.

        # Safety check: stop if the board is full.
        if len(game.move_log) >= game.rows * game.columns:
            break

    # Determine the winner.
    # We assume that when a winning move is made, game.game_over is set.
    if game.move_log:
        # The last move recorded is assumed to be the winning move.
        winning_player = game.move_log[-1]["player"]
        # Map the winning player to the corresponding bot ID.
        if ordering == "normal":
            winner_bot = bot1_id if winning_player == 1 else bot2_id
        else:
            winner_bot = bot1_id if winning_player == 1 else bot2_id
    else:
        winner_bot = "Draw"

    # Build the game record with metadata.
    game_record = {
        "metadata": {
            "bot1": bot1_id,
            "bot2": bot2_id,
            "game_number": game_number,
            "ordering": ordering,  # Indicates which bot was assigned Player 1.
            "result": winner_bot
        },
        "move_log": game.move_log
    }
    return game_record

def simulate_pairing(bot1_name, bot2_name, bot1_class, bot2_class, simulation_dir):
    """
    Simulates all games for a specific bot pairing, saving the game logs.
    """
    total_games = 0
    for game_index in range(1, 51):
        record = run_game(bot1_class, bot2_class, bot1_name, bot2_name,
                          game_number=game_index, ordering="normal")
        filename = os.path.join(simulation_dir,
                                f"{bot1_name}_vs_{bot2_name}_normal_game{game_index}.json")
        with open(filename, "w") as f:
            json.dump(record, f, indent=4)
        total_games += 1

    for game_index in range(51, 101):
        record = run_game(bot1_class, bot2_class, bot1_name, bot2_name,
                          game_number=game_index, ordering="reversed")
        filename = os.path.join(simulation_dir,
                                f"{bot1_name}_vs_{bot2_name}_reversed_game{game_index}.json")
        with open(filename, "w") as f:
            json.dump(record, f, indent=4)
        total_games += 1
    return total_games



import os
import json
import random
import time
import datetime
import copy
import concurrent.futures  # Import the module for concurrency

# Assume Connect4Game, Bot classes (RandomBot, HeuristicBot, MinimaxBot, MCTSBot),
# run_game, and simulate_pairing functions are defined as before.

def run_simulation():
    """
    Runs a tournament simulation with concurrent game execution, limited to 4 processes at a time.
    """
    # Create the directory to store game logs.
    import datetime
    simulation_dir = f"simulation_game_logs_{datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
    os.makedirs(simulation_dir, exist_ok=True)

    # Mapping of bot names to their classes.
    bot_classes = {
        "RandomBot": RandomBot,
        "HeuristicBot": HeuristicBot,
        "MinimaxBot": MinimaxBot,
        "MCTSBot": MCTSBot
    }
    bot_names = list(bot_classes.keys())
    total_games = 0

    # Use a ThreadPoolExecutor or ProcessPoolExecutor to run pairings concurrently
    # Set max_workers to 4 to limit the number of concurrent processes.
    with concurrent.futures.ThreadPoolExecutor(max_workers=8) as executor:
        futures = []
        # Iterate over each distinct pairing (do not pair a bot with itself).
        for i in range(len(bot_names)):
            for j in range(i + 1, len(bot_names)):
                bot1_name = bot_names[i]
                bot2_name = bot_names[j]
                bot1_class = bot_classes[bot1_name]
                bot2_class = bot_classes[bot2_name]

                # Submit the task to the executor
                future = executor.submit(simulate_pairing, bot1_name, bot2_name, bot1_class, bot2_class, simulation_dir)
                futures.append(future)

        # Wait for all tasks to complete and accumulate the results.
        for future in concurrent.futures.as_completed(futures):
            try:
                total_games += future.result()  # Get the number of games played by the pairing
            except Exception as e:
                print(f"A task generated an exception: {e}")

    print(f"Simulation completed. Total games played: {total_games}")

if __name__ == "__main__":
    run_simulation()


Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 2 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 wins!
Player 1 w

In [3]:
import os
import json
import pandas as pd

def create_dataframe_from_json_logs(directory="/content/simulation_game_logs_20250218_041219"):
    """
    Loads JSON game logs from a directory, parses the data, and creates a Pandas DataFrame.

    Args:
        directory (str): The directory containing the JSON game log files.  Defaults to "simulation_game_logs".

    Returns:
        pandas.DataFrame: A DataFrame containing the parsed game log data, or None if no files are found or an error occurs.
    """
    all_data = []  # List to store data from all game logs

    try:
        # Iterate over each file in the specified directory
        for filename in os.listdir(directory):
            if filename.endswith(".json"):
                filepath = os.path.join(directory, filename)
                try:
                    with open(filepath, 'r') as f:
                        game_record = json.load(f)

                    # Extract metadata
                    metadata = game_record['metadata']
                    bot1 = metadata['bot1']
                    bot2 = metadata['bot2']
                    game_number = metadata['game_number']
                    ordering = metadata['ordering']
                    result = metadata['result']

                    # Iterate over each move in the move_log
                    for move in game_record['move_log']:
                        move_number = move['move_number']
                        player = move['player']
                        column = move['column']
                        row = move['row']
                        timestamp = move['timestamp']

                        # Create a dictionary for the move data, including metadata
                        move_data = {
                            'filename': filename,  # Add the filename for reference
                            'bot1': bot1,
                            'bot2': bot2,
                            'game_number': game_number,
                            'ordering': ordering,
                            'result': result,
                            'move_number': move_number,
                            'player': player,
                            'column': column,
                            'row': row,
                            'timestamp': timestamp
                        }
                        all_data.append(move_data)

                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON in file {filename}: {e}")
                except Exception as e:
                    print(f"Error processing file {filename}: {e}")


        # Create the DataFrame from the collected data
        if all_data:
            df = pd.DataFrame(all_data)
            return df
        else:
            print("No game log data found.")
            return None  # Return None if no data was collected

    except FileNotFoundError:
        print(f"Directory not found: {directory}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage:
if __name__ == "__main__":
    df = create_dataframe_from_json_logs()
    if df is not None:
        print(df.head())  # Print the first few rows of the DataFrame
        print(df.info())   # Print summary information about the DataFrame (data types, etc.)

        # Example analysis (after creating the DataFrame):
        print("\nNumber of games played by each bot pairing:")
        print(df.groupby(['bot1', 'bot2']).size())  # Count games per pairing

        print("\nWin rate of each bot as Player 1:")
        player1_wins = df[(df['player'] == 1) & (df['result'] != 'Draw')].groupby('bot1')['result'].apply(lambda x: (x == x.name).sum() / len(x))
        print(player1_wins)

                                   filename        bot1     bot2  game_number  \
0  MinimaxBot_vs_MCTSBot_normal_game35.json  MinimaxBot  MCTSBot           35   
1  MinimaxBot_vs_MCTSBot_normal_game35.json  MinimaxBot  MCTSBot           35   
2  MinimaxBot_vs_MCTSBot_normal_game35.json  MinimaxBot  MCTSBot           35   
3  MinimaxBot_vs_MCTSBot_normal_game35.json  MinimaxBot  MCTSBot           35   
4  MinimaxBot_vs_MCTSBot_normal_game35.json  MinimaxBot  MCTSBot           35   

  ordering      result  move_number  player  column  row  \
0   normal  MinimaxBot            1       1       3    5   
1   normal  MinimaxBot            2       2       3    4   
2   normal  MinimaxBot            3       1       3    3   
3   normal  MinimaxBot            4       2       4    5   
4   normal  MinimaxBot            5       1       4    4   

                    timestamp  
0  2025-02-18T04:41:25.236715  
1  2025-02-18T04:41:27.237135  
2  2025-02-18T04:41:29.241595  
3  2025-02-18T04:41:31.2

In [ ]:
df

,filename,bot1,bot2,game_number,ordering,result,move_number,player,column,row,timestamp
0,MinimaxBot_vs_MCTSBot_normal_game35.json,MinimaxBot,MCTSBot,35,normal,MinimaxBot,1,1,3,5,2025-02-17T16:02:13.826859
1,MinimaxBot_vs_MCTSBot_normal_game35.json,MinimaxBot,MCTSBot,35,normal,MinimaxBot,2,2,3,4,2025-02-17T16:02:15.827185
2,MinimaxBot_vs_MCTSBot_normal_game35.json,MinimaxBot,MCTSBot,35,normal,MinimaxBot,3,1,3,3,2025-02-17T16:02:17.827332
3,MinimaxBot_vs_MCTSBot_normal_game35.json,MinimaxBot,MCTSBot,35,normal,MinimaxBot,4,2,2,5,2025-02-17T16:02:19.827425
4,MinimaxBot_vs_MCTSBot_normal_game35.json,MinimaxBot,MCTSBot,35,normal,MinimaxBot,5,1,3,2,2025-02-17T16:02:21.827557
...,...,...,...,...,...,...,...,...,...,...,...
8709,MinimaxBot_vs_MCTSBot_normal_game3.json,MinimaxBot,MCTSBot,3,normal,MinimaxBot,19,1,4,5,2025-02-17T15:37:03.796781
8710,MinimaxBot_vs_MCTSBot_normal_game3.json,MinimaxBot,MCTSBot,3,normal,MinimaxBot,20,2,2,1,2025-02-17T15:37:05.804546
8711,MinimaxBot_vs_MCTSBot_normal_game3.json,MinimaxBot,MCTSBot,3,normal,MinimaxBot,21,1,4,4,2025-02-17T15:37:07.804656
8712,MinimaxBot_vs_MCTSBot_normal_game3.json,MinimaxBot,MCTSBot,3,normal,MinimaxBot,22,2,0,5,2025-02-17T15:37:09.804797
